In [1]:
from langchain.embeddings import OpenAIEmbeddings
from langchain.embeddings.huggingface import HuggingFaceEmbeddings
import numpy as np

#from ray.data import ActorPoolStrategy

class EmbedChunks:
    def __init__(self, model_name):
        if model_name == "text-embedding-ada-002":
            self.embedding_model = OpenAIEmbeddings(
                model=model_name,
                openai_api_base=os.environ["OPENAI_API_BASE"],
                openai_api_key=os.environ["OPENAI_API_KEY"])
        else:
            self.embedding_model = HuggingFaceEmbeddings(
                model_name=model_name,
                model_kwargs={"device": "cuda"},
                encode_kwargs={"device": "cuda", "batch_size": 100})

    def __call__(self, batch):
        embeddings = self.embedding_model.embed_documents(batch["text"])
        return {"question": batch["question"], "solution": batch["solution"], "embeddings": 
embeddings}

In [8]:
from dotenv import load_dotenv
import os
load_dotenv()

openai_api_key = os.getenv("OPENAI_API_KEY")
from langchain.embeddings.openai import OpenAIEmbeddings
import pandas as pd
data_path = "../dataset/data.jsonl"

df = pd.read_json(data_path, lines=True)
embed_model = OpenAIEmbeddings(model="text-embedding-ada-002",openai_api_key=openai_api_key)
texts = df['question'] + '' + df['solution']




embeded_chunk = embed_model.embed_documents(texts)
len(embeded_chunk), len(embeded_chunk[0])
#for i, embeded_chunk in enumerate(embeded_chunk):
    #print(f"Embedded representation for chunk {i+1}:")
    #print(embeded_chunk)



(4253, 1536)

In [11]:
class StoreResults:
    def __call__(self, batch):
        # Establish a connection to the database using the DB_CONNECTION_STRING from the environment variables
        with psycopg.connect(os.environ["DB_CONNECTION_STRING"]) as conn:
            # Register the vector type in the database (assuming it's a custom vector type)
            register_vector(conn)
            with conn.cursor() as cur:
                # Iterate over each element in the batch and insert the data into the database
                for text, source, embedding in zip(batch["text"], batch["source"], batch["embeddings"]):
                    cur.execute("INSERT INTO document (text, source, embedding) VALUES (%s, %s, %s)",
                                (text, source, embedding,))
        # Return an empty dictionary
        return {}
        
embeded_chunk.map_batches(
    StoreResults,
    batch_size=128,
    num_cpus=1,
    compute=ActorPoolStrategy(size=28),
).count()


AttributeError: 'list' object has no attribute 'map_batches'

In [12]:
print(type(embeded_chunk))

<class 'list'>
